<a href="https://colab.research.google.com/github/dtoralg/IE_Calidad_ML/blob/main/Ejercicios/Manual_Practico_Programacion_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Manual Práctico de Machine Learning con Python

Este cuaderno contiene una guía completa para aprender y practicar los principales conceptos y técnicas de Machine Learning con Python. Está organizado por bloques temáticos. Cada bloque incluye:

- Una **explicación breve**
- Un **ejemplo ejecutable**
- Un **ejercicio práctico guiado**
- Una **reflexión sobre para qué sirve y cuándo se usa**

> **Sigue el orden o navega por el índice según tus intereses.**


### Consejo importante para aprender y mejorar tu programación

No intentes memorizar el código.

**Tu objetivo no es recordar cada función o parámetro, sino entender qué hace cada bloque**.

- ¿Por qué se usa `ReLU` en una capa oculta?
- ¿Qué significa `categorical_crossentropy` y cuándo se usa?
- ¿Por qué usamos `softmax` en la salida para clasificación multiclase?

Estas preguntas valen más que repetir código de memoria.

#### Recomendaciones:

- Si no recuerdas la sintaxis: **búscala o usa ejemplos anteriores**.
- Si ves una función nueva: **lee su descripción** o prueba con `help(función)` o `Shift + Tab` si usas Jupyter Notebooks.
- Si algo no funciona: **imprime variables, revisa formas (`.shape`), ejecuta por partes**.
- Si te bloqueas: **comparte lo que estás intentando hacer, no solo el error**.

**Aprender a programar es como aprender a hablar otro idioma: necesitas práctica, contexto y repetición. La memoria vendrá después.**

## 1. Carga y Exploración de Datos

### ¿Qué es?

Cargar y explorar datos es el primer paso de cualquier proceso de análisis. Aquí es donde obtenemos una primera impresión de cómo están estructurados los datos, qué tipo de variables tenemos, si hay valores nulos, columnas irrelevantes, etc.

Esto se conoce también como **ETL (Extract, Transform, Load)** en entornos más profesionales.



### 1.1 Cargar un archivo CSV

**Código de ejemplo:**  
Cargamos un dataset desde un archivo CSV local o una URL.


In [ ]:
import pandas as pd

# Cargar archivo desde local
df = pd.read_csv("datos_calidad.csv")  # Carga un archivo CSV en un DataFrame de pandas
# También se puede usar una URL si el archivo está en línea
# df = pd.read_csv("https://ruta-al-archivo/dataset.csv")

df.head()  # Muestra las primeras filas del DataFrame

**Ejercicio guiado:**  
Cambia el nombre del archivo anterior por otro CSV disponible en tu equipo o entorno, y muestra las 10 primeras filas usando `.head(10)`.

En Colab, puedes hacer click en la barra lateral izquierda, en el icono "Archivos" y subir ahí tus datos desde el PC o desde Drive para que estén disponibles en este entorno. Cuando reinicies el entorno desaparecerán.

Una vez tengas el archivo subido, te recomiendo hacer click derecho > "Copiar Ruta". Por ejemplo, tendrías `pd.read_csv("/content/sample_data/mnist_train_small.csv
")`



**¿Para qué sirve?**  
Este paso es fundamental para poder trabajar con datos reales. Sin esta carga inicial no es posible iniciar ningún análisis, ni visualizaciones ni modelos.


In [ ]:
# sklearn pone a nuestra disposición una serie de datasets muy útiles para hacer pruebas
# https://scikit-learn.org/stable/datasets/toy_dataset.html

from sklearn.datasets import load_iris, load_breast_cancer  # Importa datasets clásicos de sklearn
import pandas as pd  # Librería para manipular datos en formato tabla

# Cargar el dataset Iris
# Este dataset contiene 150 flores con 4 características cada una (largo/ancho de pétalo y sépalo)
iris = load_iris()

# Convertir los datos en un DataFrame de Pandas
# - iris.data → matriz con las variables numéricas
# - iris.feature_names → nombres de las columnas (características)
df_iris = pd.DataFrame(iris.data, columns=iris.feature_names)

# Añadir la columna 'target' que contiene la especie (0, 1 o 2)
df_iris["target"] = iris.target

# Mostrar las primeras filas del DataFrame
df_iris.head()


### 1.2 Exploración inicial del dataset

In [ ]:
# Ver forma del dataset
print("Dimensiones:", df.shape)

# Tipos de datos
print(df.dtypes)

# Resumen estadístico
df.describe()

**Explicación:**  
- `.shape` nos dice cuántas filas y columnas hay.
- `.dtypes` muestra el tipo de cada columna.
- `.describe()` da estadísticas básicas como media, desviación, mínimo y máximo para columnas numéricas.


### 1.3 Comprobación de valores faltantes

In [ ]:
# Comprobar nulos por columna
df.isnull().sum().sort_values(ascending=False).head(10) # Nos devuelve las 10 columnas con más valores faltantes del DataFrame

**¿Para qué sirve?**  
Muchas funciones y modelos no admiten valores nulos, por lo que es importante identificarlos y decidir si se imputan (rellenan) o se eliminan.


### 1.4 Eliminar columnas irrelevantes

In [ ]:
# Supongamos que hay una columna 'ID' que no aporta valor predictivo
if 'ID' in df.columns:  # Lista los nombres de las columnas
    df = df.drop(columns=['ID'])

**Ejercicio guiado:**  
Busca si tu dataset contiene columnas como identificadores, fechas de carga u observaciones constantes y elimínalas del análisis ya que no aportan capacidad predictiva y pueden inferir errores en tus modelos (relaciones espúreas).


### 1.5 Matriz de correlación

In [ ]:
# Visualizar las correlaciones entre variables numéricas
plt.figure(figsize=(10, 8))
sns.heatmap(df[num_cols].corr(), annot=True, cmap='coolwarm')
plt.title("Matriz de correlación entre variables numéricas")
plt.show()

### 1.6 Visualizacion de la distrubución de clases

In [ ]:
# Ver la cantidad de ejemplos por clase en una variable categórica (por ejemplo, 'Estado')
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(x='Estado_cod', data=df)
plt.title("Distribución de clases en la variable 'Estado'")
plt.xlabel("Clase")
plt.ylabel("Cantidad de ejemplos")
plt.grid(True)
plt.show()

## 2. Preparación de Datos

Antes de entrenar cualquier modelo, es fundamental preparar los datos correctamente.  
Este proceso incluye:

- **Escalar y normalizar** los datos numéricos
- **Codificar** variables categóricas
- Aplicar técnicas de **feature engineering**
- Preparar los datos para evitar fugas de información y mejorar la generalización


### 2.1 Escalado de variables numéricas

In [ ]:
from sklearn.preprocessing import StandardScaler  # Importa el escalador estándar de scikit-learn

# Selección de columnas numéricas del DataFrame
num_cols = df.select_dtypes(include='number').columns  # Detecta automáticamente las columnas numéricas

# Crear el escalador:
# - centrará las variables (media = 0)
# - escalará para que tengan desviación estándar = 1
scaler = StandardScaler()

# Hacemos una copia del DataFrame original para mantener los datos sin modificar
df_scaled = df.copy()

# Aplicamos el escalado solo a las columnas numéricas
# fit_transform: ajusta el escalador y transforma los datos
df_scaled[num_cols] = scaler.fit_transform(df[num_cols])

# Visualizamos las primeras filas del DataFrame escalado
df_scaled.head()


**¿Por qué escalar?**  
Muchos modelos (KNN, Regresión Logística, Redes Neuronales) son sensibles a la escala de las variables.  
El `StandardScaler` transforma cada variable para que tenga media 0 y desviación estándar 1.


### 2.2 Codificación de variables categóricas

In [ ]:
from sklearn.preprocessing import LabelEncoder  # Importa el codificador de etiquetas

# Suponemos que 'Estado' es la variable objetivo categórica
if 'Estado' in df.columns:  # Verificamos que la columna existe en el DataFrame
    le = LabelEncoder()  # Crea una instancia del codificador LabelEncoder
    df['Estado_cod'] = le.fit_transform(df['Estado'])  # Codifica las categorías como números enteros
    print("Clases:", list(le.classes_))  # Muestra las clases originales asignadas a cada número
    df[['Estado', 'Estado_cod']].head()  # Muestra una vista de las etiquetas originales y su codificación


**¿Para qué sirve?**  
Muchos modelos de machine learning necesitan que las variables categóricas estén en formato numérico.  
`LabelEncoder` convierte etiquetas como `"OK"`, `"KO"` en `0`, `1`.


### 2.3 Ingeniería de características (Feature Engineering)

In [ ]:
# Crear una nueva variable: relación entre temperatura y presión
df['Temp_Pres_ratio'] = df['Temperatura'] / df['Presion'] # Creamos una nueva variable en df lamada Temp_Pres_Ratio
df[['Temperatura', 'Presion', 'Temp_Pres_ratio']].head()  # Muestra las primeras filas del DataFrame

**¿Para qué sirve?**  
A veces, relaciones entre variables aportan más valor predictivo que las variables originales por separado.


### 2.4 Eliminar columnas constantes o duplicadas

In [ ]:
# Eliminar columnas que tienen un único valor (constantes)
for col in df.columns:  # Recorre cada columna del DataFrame
    if df[col].nunique() == 1:  # Si solo tiene un valor único (sin variabilidad)
        df = df.drop(columns=[col])  # Elimina esa columna del DataFrame

**¿Para qué sirve?**  
Las columnas que no cambian no aportan información y pueden dificultar el entrenamiento o inflar el tamaño del modelo innecesariamente.


### 2.5 Codificación de variables categóricas (OneHotEncoder)

In [ ]:
# Convertir variables categóricas en columnas binarias (0/1)
df_ohe = pd.get_dummies(df, columns=['Categoria'], drop_first=True)  # 'Categoria' debe reemplazarse por tu columna categórica
df_ohe.head()

**¿Para qué sirve?**  
- Convierte **variables categóricas** (como "Color", "Estado", "Tipo") en **columnas numéricas binarias** (0/1), que los modelos pueden entender.
- Cada categoría se convierte en una nueva columna.
- Permite usar **textos como entradas** en modelos de machine learning sin errores.

## 3. Modelos Supervisados

### 3.1 Regresión Lineal

In [ ]:
from sklearn.linear_model import LinearRegression  # Importa el modelo de regresión lineal

# Seleccionamos una sola variable como predictor: 'Temperatura'
# y una variable objetivo continua: 'Presion'
X_simple = df[["Temperatura"]]  # Debe ser un DataFrame (matriz 2D)
y_simple = df["Presion"]        # Variable de salida (1D)

# Crear el modelo de regresión lineal
model_lr = LinearRegression()

# Entrenar el modelo ajustando una recta (y = a*x + b) a los datos
model_lr.fit(X_simple, y_simple)

# Predecir los valores de presión a partir de la temperatura
# Las predicciones estarán en la misma escala que y_simple
y_pred_lr = model_lr.predict(X_simple)



**¿Para qué sirve?**  
La regresión lineal permite predecir un valor numérico continuo.  
Es útil como modelo de referencia y también para interpretar relaciones lineales entre variables.


### 3.2 Regresión Logística

In [ ]:
from sklearn.linear_model import LogisticRegression  # Importa el modelo de regresión logística

# Usamos como entrada las columnas numéricas escaladas
# X_log: variables predictoras
# y_log: variable objetivo binaria (0 o 1)
X_log = df_scaled[num_cols]
y_log = df["Estado_cod"]

# Crear el modelo de regresión logística
# - max_iter=1000: número máximo de iteraciones para que el algoritmo converja
model_log = LogisticRegression(max_iter=1000)

# Entrenar el modelo con los datos completos
# El modelo ajusta una función sigmoide para predecir la probabilidad de clase 1
model_log.fit(X_log, y_log)

# Hacer predicciones sobre los mismos datos (en práctica, usarías X_test)
# El resultado es una clase predicha (0 o 1)
y_pred_log = model_log.predict(X_log)


**¿Para qué sirve?**  
Clasifica instancias en dos clases (binaria) de forma simple pero efectiva.  
Suele ser el modelo base para comparar con otros más complejos.


### 3.3 Árbol de Decisión

In [ ]:
from sklearn.tree import DecisionTreeClassifier  # Importa el modelo de árbol de decisión

# Crear el modelo:
# - max_depth=4: limita la profundidad máxima del árbol a 4 niveles (para evitar sobreajuste)
model_dt = DecisionTreeClassifier(max_depth=4)

# Entrenar el modelo con los datos de entrenamiento
# El árbol aprende reglas de decisión (si... entonces...) a partir de los datos
model_dt.fit(X_train_log, y_train_log)

# Realizar predicciones sobre los datos de prueba
# El modelo recorre las ramas del árbol para clasificar cada muestra
y_pred_dt = model_dt.predict(X_test_log)


**¿Para qué sirve?**  
Los árboles son fáciles de interpretar y permiten entender reglas de decisión.  
Son útiles cuando hay relaciones no lineales entre las variables.


### 3.4 Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier  # Importa el clasificador Random Forest

# Crear el modelo:
# - n_estimators=100: usa 100 árboles de decisión independientes
# - random_state=42: fija la semilla para que los resultados sean reproducibles
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo con los datos de entrenamiento
# Cada árbol se entrena con una muestra aleatoria distinta del dataset (bagging)
model_rf.fit(X_train_log, y_train_log)

# Realizar predicciones sobre los datos de prueba
# El resultado final es el voto mayoritario entre los árboles del bosque
y_pred_rf = model_rf.predict(X_test_log)


**¿Para qué sirve?**  
Es un ensemble de árboles que mejora la robustez y generalización del modelo.  
Muy usado en la industria por su rendimiento y facilidad de uso.

Cuando el modelo lo permite, es recomendable fijar `random_state` para que el resultado sea repetible.



In [ ]:
# Obtener la importancia de cada variable es muy útil en modelos basados en árboles (como Random Forest)
import pandas as pd
import matplotlib.pyplot as plt

# Crear una serie de Pandas con la importancia de cada variable
# model_rf.feature_importances_ devuelve un array con un valor por cada variable de entrada
# Usamos los nombres de las columnas como índice
importancia = pd.Series(model_rf.feature_importances_, index=X_train_log.columns)

# Ordenamos las variables por importancia y creamos un gráfico de barras horizontal
importancia.sort_values().plot(kind='barh')

# Personalizamos el gráfico
plt.title("Importancia de cada variable (Random Forest)")
plt.xlabel("Importancia")
plt.grid(True)
plt.show()  # Muestra el gráfico generado


### 3.5 K-Nearest Neighbors (KNN)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier  # Importa el clasificador KNN

# Crear el modelo KNN:
# - n_neighbors=5: el modelo clasificará en función de los 5 vecinos más cercanos
model_knn = KNeighborsClassifier(n_neighbors=5)

# Entrenar el modelo con los datos de entrenamiento
# El modelo memoriza todos los puntos de entrenamiento (no ajusta parámetros)
model_knn.fit(X_train_log, y_train_log)

# Hacer predicciones sobre los datos de prueba
# Para cada punto nuevo, el modelo mira sus 5 vecinos más cercanos y predice la clase más frecuente
y_pred_knn = model_knn.predict(X_test_log)


**¿Para qué sirve?**  
Clasifica en base a los vecinos más cercanos.  
Muy intuitivo y efectivo en datasets pequeños y bien escalados.


## 4. Modelos No Supervisados

Los modelos no supervisados aprenden a partir de datos **sin etiquetas**. Se utilizan para:
- Explorar la estructura interna de los datos
- Agrupar observaciones similares
- Reducir la dimensionalidad para visualización o mejora de modelos supervisados


### 4.1 K-Means - Agrupamiento no supervisado

In [ ]:
from sklearn.cluster import KMeans  # Importa el algoritmo K-Means

# Aplicar KMeans con 2 grupos (clusters)
# - n_clusters=2: número de agrupaciones a encontrar
# - random_state=42: para que el resultado sea reproducible
kmeans = KMeans(n_clusters=2, random_state=42)

# Ajusta el modelo y predice el cluster asignado para cada muestra
clusters = kmeans.fit_predict(X)

# Visualizar los clusters si X tiene solo 2 variables (2D)
plt.scatter(X[:, 0], X[:, 1], c=clusters, cmap='Set2')  # Pinta cada punto según el cluster asignado

# Añadir los centroides (centros de los clusters) como marcadores 'X' negros
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
            c='black', s=200, alpha=0.7, marker='X', label='Centroides')

# Personalizar el gráfico
plt.title("Clusters detectados por K-Means")
plt.xlabel("Variable 1")
plt.ylabel("Variable 2")
plt.legend()
plt.grid(True)
plt.show()


**¿Para qué sirve?**  
K-Means permite **agrupar observaciones similares** sin necesidad de etiquetas.  
Se usa en segmentación de clientes, análisis exploratorio y detección de patrones no etiquetados.


### 4.1 PCA - Análisis de Componentes Principales

In [ ]:
from sklearn.decomposition import PCA  # Importa el modelo de análisis de componentes principales
import matplotlib.pyplot as plt        # Librería para graficar

# Aplicar PCA para reducir los datos a 2 dimensiones (componentes principales)
# Esto nos permite visualizar datos con muchas variables en un plano 2D
pca = PCA(n_components=2)

# Ajusta el PCA sobre X_log y transforma los datos a 2 componentes
X_pca = pca.fit_transform(X_log)

# Crear un gráfico de dispersión con las nuevas dimensiones
# c=y_log → colorea cada punto según su clase original (si hay etiquetas disponibles)
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_log, cmap='coolwarm', alpha=0.7)

# Personalizar el gráfico
plt.xlabel("Componente 1")
plt.ylabel("Componente 2")
plt.title("Visualización PCA (2D)")
plt.grid(True)
plt.show()  # Muestra el gráfico


**¿Para qué sirve?**  
PCA transforma las variables originales en un nuevo conjunto de variables **no correlacionadas**, que capturan la mayor parte de la varianza.
Se usa para visualización, compresión o preprocesamiento antes de modelos.


## 5. Evaluación de Modelos

Evaluar el rendimiento de un modelo es tan importante como entrenarlo.  
Dependiendo del tipo de problema (clasificación o regresión), usaremos diferentes métricas:

- Clasificación: precisión, recall, F1, matriz de confusión, ROC, AUC
- Regresión: MAE, MSE, RMSE, R²


### 5.1 Matriz de Confusión

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay  # Importa funciones para crear y mostrar la matriz de confusión

# Calcula la matriz de confusión a partir de los valores reales y los predichos
cm = confusion_matrix(y_test_log, y_pred_rf)

# Crea el objeto de visualización a partir de la matriz
disp = ConfusionMatrixDisplay(confusion_matrix=cm)

# Muestra la matriz con un mapa de colores azul
disp.plot(cmap='Blues')

# Añade un título y muestra el gráfico
plt.title("Matriz de Confusión")
plt.show()


**¿Para qué sirve?**  
Muestra el número de aciertos y errores para cada clase.  
Ideal para saber **qué clases se confunden entre sí**.


### 5.2 Precisión, Recall, F1 Score, Accuracy

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score  # Importa métricas de evaluación

# Accuracy: porcentaje de predicciones correctas (aciertos totales)
print("Accuracy:", accuracy_score(y_test_log, y_pred_rf))

# Precisión: de todas las veces que se predijo clase positiva, ¿cuántas eran correctas?
print("Precisión:", precision_score(y_test_log, y_pred_rf))

# Recall: de todas las verdaderas instancias positivas, ¿cuántas detectamos?
print("Recall:", recall_score(y_test_log, y_pred_rf))

# F1 Score: media armónica entre precisión y recall (balance entre ambos)
print("F1 Score:", f1_score(y_test_log, y_pred_rf))

**¿Cuándo usar cada una?**
- `Precisión`: cuántos de los positivos predichos eran correctos
- `Recall`: cuántos de los positivos reales fueron capturados
- `F1`: equilibrio entre precisión y recall
- `Accuracy`: proporción de aciertos totales (menos útil si hay clases desbalanceadas)


### 5.3 Curva ROC y AUC

In [ ]:
from sklearn.metrics import roc_curve, auc  # Importa funciones para calcular curva ROC y el área bajo la curva (AUC)

# Obtener las probabilidades predichas para la clase positiva (índice 1)
# predict_proba devuelve [P(clase 0), P(clase 1)], nos interesa la de clase 1
y_prob = model_rf.predict_proba(X_test_log)[:, 1]

# Calcular los puntos de la curva ROC:
# - fpr: tasa de falsos positivos
# - tpr: tasa de verdaderos positivos
# - thresholds: umbrales usados para clasificar
fpr, tpr, thresholds = roc_curve(y_test_log, y_prob)

# Calcular el área bajo la curva ROC (AUC)
roc_auc = auc(fpr, tpr)

# Dibujar la curva ROC
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")  # Curva ROC
plt.plot([0, 1], [0, 1], linestyle='--')  # Línea diagonal (modelo aleatorio)

# Etiquetas del gráfico
plt.xlabel("Tasa de falsos positivos (FPR)")
plt.ylabel("Tasa de verdaderos positivos (TPR)")
plt.title("Curva ROC")
plt.legend()
plt.grid(True)
plt.show()  # Muestra el gráfico generado


**¿Para qué sirve?**  
Mide la capacidad del modelo para distinguir entre clases.  
Cuanto más se acerque el AUC a 1, mejor.


### 5.4 MAE, MSE, RMSE, R²

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score  # Importa métricas de regresión
import numpy as np  # Importa numpy para calcular la raíz cuadrada

# Evaluamos el modelo de regresión lineal previamente entrenado

# MAE (Mean Absolute Error): promedio de los errores absolutos
mae = mean_absolute_error(y_simple, y_pred_lr)

# MSE (Mean Squared Error): promedio de los errores al cuadrado
mse = mean_squared_error(y_simple, y_pred_lr)

# RMSE: raíz cuadrada del MSE (misma unidad que la variable original)
rmse = np.sqrt(mse)

# R² (Coeficiente de determinación): qué proporción de la varianza de y es explicada por el modelo
r2 = r2_score(y_simple, y_pred_lr)

# Mostrar los resultados con dos decimales
print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.2f}")


**¿Qué mide cada métrica?**
- `MAE`: error absoluto medio
- `MSE`: error cuadrático medio (penaliza más errores grandes)
- `RMSE`: raíz cuadrada del MSE (en mismas unidades que la variable)
- `R²`: porcentaje de variabilidad explicada por el modelo


## 6. Técnicas de Optimización y Regularización

Estas técnicas ayudan a mejorar el rendimiento y la generalización del modelo, especialmente cuando los datos son complejos o limitados.


### 6.1 Regularización L1 y L2

In [ ]:
from sklearn.linear_model import LogisticRegression  # Importa el modelo de regresión logística

# Modelo con regularización L1 (Lasso):
# - penalty='l1': aplica regularización L1 (algunos coeficientes se vuelven exactamente cero)
# - solver='liblinear': necesario para usar L1 en problemas pequeños o binarios
model_l1 = LogisticRegression(penalty='l1', solver='liblinear')
model_l1.fit(X_train_log, y_train_log)  # Entrena el modelo con regularización L1

# Modelo con regularización L2 (Ridge):
# - penalty='l2': aplica regularización L2 (reduce la magnitud de los coeficientes)
# - solver='lbfgs' se usa por defecto y es compatible con L2
model_l2 = LogisticRegression(penalty='l2')
model_l2.fit(X_train_log, y_train_log)  # Entrena el modelo con regularización L2


**¿Para qué sirve?**  
- `L1`: fuerza coeficientes a cero (selección de variables)
- `L2`: reduce la magnitud de los coeficientes (reduce overfitting)


### 6.2 Búsqueda de hiperparámetros con GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV  # Importa GridSearchCV para optimización de hiperparámetros

# Definir la rejilla de hiperparámetros que queremos probar:
# - 'n_estimators': número de árboles en el bosque
# - 'max_depth': profundidad máxima de cada árbol
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5, 10]
}

# Crear la búsqueda con validación cruzada:
# - modelo base: RandomForestClassifier
# - param_grid: rejilla de combinaciones a probar
# - cv=5: validación cruzada con 5 particiones
# - scoring='f1_macro': métrica a optimizar (media del F1 por clase)
grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='f1_macro')

# Ajustar la búsqueda al conjunto de entrenamiento
grid.fit(X_train_log, y_train_log)

# Mostrar los mejores parámetros encontrados
print("Mejores parámetros:", grid.best_params_)

# Mostrar el mejor resultado promedio de F1 (macro) durante la validación cruzada
print("Mejor score F1 Macro:", grid.best_score_)


**¿Para qué sirve?**  
Permite probar muchas combinaciones de hiperparámetros y seleccionar la mejor automáticamente.


### 6.3 RandomizedSearchCV (alternativa rápida a GridSearch)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV  # Búsqueda aleatoria de hiperparámetros con validación cruzada
from scipy.stats import randint  # Distribución de enteros aleatorios

# Definir el espacio de búsqueda para los hiperparámetros:
# - 'n_estimators': número de árboles (valores entre 50 y 150)
# - 'max_depth': profundidad del árbol (valores entre 3 y 10)
param_dist = {
    'n_estimators': randint(50, 150),
    'max_depth': randint(3, 10)
}

# Crear la búsqueda aleatoria:
# - modelo base: RandomForestClassifier
# - param_distributions: espacio de búsqueda definido arriba
# - n_iter=10: probará 10 combinaciones aleatorias
# - cv=5: validación cruzada con 5 particiones
# - random_state: para obtener resultados reproducibles
random_search = RandomizedSearchCV(RandomForestClassifier(),
                                   param_distributions=param_dist,
                                   n_iter=10,
                                   cv=5,
                                   random_state=42)

# Entrenar el modelo y buscar la mejor combinación
random_search.fit(X_train_log, y_train_log)

# Mostrar la mejor combinación encontrada
print("Mejores parámetros:", random_search.best_params_)


### 6.4 Rebalanceo de clases con SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE  # Importa la técnica SMOTE (Synthetic Minority Over-sampling Technique)
from collections import Counter  # Permite contar la frecuencia de valores en listas

# Crear el objeto SMOTE:
# - random_state=42: fija la semilla para reproducibilidad
# SMOTE genera ejemplos sintéticos nuevos para la clase minoritaria
smote = SMOTE(random_state=42)

# Aplicar SMOTE al conjunto de entrenamiento
# fit_resample ajusta y aplica el sobremuestreo
X_train_sm, y_train_sm = smote.fit_resample(X_train_log, y_train_log)

# Mostrar la distribución de clases antes y después del rebalanceo
print("Distribución original:", Counter(y_train_log))
print("Distribución balanceada:", Counter(y_train_sm))


**¿Para qué sirve?**  
Genera muestras sintéticas para la clase minoritaria y evita el sobreajuste a la clase mayoritaria.


## 6. Redes Neuronales: desde MLP hasta LSTM y CNN

Las redes neuronales permiten modelar relaciones no lineales complejas.  
En esta sección las construiremos de forma progresiva:

1. Red Neuronal Multicapa básica (MLP)
2. MLP más profunda con Dropout y activación ReLU
3. Red LSTM (para series temporales)
4. Red Convolucional (CNN, para datos con estructura espacial)
5. Uso de EarlyStopping para evitar overfitting


### 6.1 Red Neuronal Multicapa (MLP) Básica

#### Funciones de activación más comunes para MLP

**ReLU (`relu`)**  
- f(x) = max(0, x)  
- Muy usada en capas ocultas  
- Rápida y evita saturación

**Sigmoid (`sigmoid`)**  
- f(x) = 1 / (1 + exp(-x))  
- Salida entre 0 y 1  
- Útil en clasificación binaria (salida)

**Softmax (`softmax`)**  
- Devuelve probabilidades que suman 1  
- Usada en clasificación multiclase (salida)

**Lineal (`linear`)**  
- f(x) = x  
- Se usa en regresión (salida continua)


### 6.1.1 Red Neuronal Multicapa para Regresión


#### Funciones de pérdida más comunes para regresión

- **`mean_squared_error` (MSE)**  
  Penaliza fuertemente los errores grandes. Es la más usada por defecto.  
  `model.compile(optimizer='adam', loss='mean_squared_error')`

- **`mean_absolute_error` (MAE)**  
  Más robusta a outliers. Penaliza todos los errores por igual.  
  `model.compile(optimizer='adam', loss='mean_absolute_error')`

- **`huber_loss`**  
  Combina MSE y MAE: usa MSE para errores pequeños y MAE para errores grandes.  
  `model.compile(optimizer='adam', loss='huber_loss')`

Este bloque muestra cómo usar una red neuronal para predecir valores numéricos continuos (por ejemplo: precio, temperatura, producción, etc.), en lugar de clasificar clases.

· Última capa: solo tiene 1 neurona

· Activación final: sin activación (linear)

· Función de pérdida: usamos mean_squared_error en lugar de categorical_crossentropy

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense  # Capa densamente conectada (fully connected), típica en MLP

# Crear un modelo secuencial: las capas se añaden en orden, una tras otra
model_reg = Sequential()

# Primera capa oculta con 64 neuronas y activación ReLU
# input_shape define el número de variables de entrada
model_reg.add(Dense(64, activation='relu', input_shape=(X_train_reg.shape[1],)))

# Segunda capa oculta con 32 neuronas y activación ReLU
model_reg.add(Dense(32, activation='relu'))

# Capa de salida con 1 sola neurona (predicción escalar continua)
# No se usa función de activación → salida lineal (ideal para regresión)
model_reg.add(Dense(1))

# Compilar el modelo:
# - Optimizador: Adam, muy usado por su estabilidad y buen rendimiento
# - Pérdida: mean_squared_error (error cuadrático medio, típico en regresión)
# - Métrica: mean_absolute_error para tener una idea clara del error medio en unidades originales
model_reg.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Entrenar el modelo:
# - validation_split=0.2: reserva el 20% de los datos de entrenamiento para validación interna
# - epochs=100: número de pasadas completas sobre los datos
# - batch_size=32: número de muestras que se procesan antes de actualizar los pesos
# - verbose=0: no muestra el progreso durante el entrenamiento (usa 1 o 2 si se quiere visualizar)
history_reg = model_reg.fit(X_train_reg, y_train_reg,
                            validation_split=0.2,
                            epochs=100,
                            batch_size=32,
                            verbose=0)

print("Entrenamiento completado.")

# Predecimos los valores continuos sobre los datos de test
y_pred_reg = model_reg.predict(X_test_reg)  # Devuelve un array de valores predichos

print("Primeras predicciones:", np.round(y_pred_reg[:5].flatten(), 2))  # Redondeamos para visualización
print("Valores reales:", np.round(y_test_reg[:5].values, 2))

### 6.1.2 Red Neuronal Multicapa para Clasificación


#### Funciones de pérdida más comunes en clasificación

**`categorical_crossentropy`**  
- Para clasificación multiclase con one-hot encoding  
- Se usa cuando la salida tiene varias clases y está codificada como vector  
- `model.compile(optimizer='adam', loss='categorical_crossentropy')`

**`sparse_categorical_crossentropy`**  
- Igual que `categorical_crossentropy`, pero con etiquetas como enteros en lugar de one-hot  
- Más cómoda si no usas `to_categorical()`  
- `model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')`

**Código explicado paso a paso:**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense  # Crea una capa densa (fully connected)
from tensorflow.keras.utils import to_categorical

# Codificamos las etiquetas enteras como vectores one-hot (necesario para usar softmax + categorical_crossentropy)
y_train_cat = to_categorical(y_train_log)
y_test_cat = to_categorical(y_test_log)

# Crear el modelo secuencial: se añaden las capas en orden, una tras otra
model_basic = Sequential()

# Primera (y única) capa oculta con 16 neuronas y activación ReLU
# input_shape define el número de características de entrada
model_basic.add(Dense(16, activation='relu', input_shape=(X_train_log.shape[1],)))

# Capa de salida con 2 neuronas (porque tenemos 2 clases) y activación softmax
# Softmax convierte las salidas en probabilidades que suman 1
model_basic.add(Dense(2, activation='softmax'))

# Compilar el modelo:
# - Optimizador Adam ajusta automáticamente los pesos
# - categorical_crossentropy porque usamos one-hot encoding
# - Métrica de evaluación: accuracy
model_basic.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
# - validation_split=0.2: usa el 20% de los datos para validación
# - epochs=100: número de veces que se recorren todos los datos de entrenamiento
# - batch_size=32: número de muestras usadas antes de actualizar los pesos
# - verbose=0: silencia la salida (puedes usar 1 o 2 para ver el progreso)
history_basic = model_basic.fit(X_train_log, y_train_cat,
                                validation_split=0.2,
                                epochs=100,
                                batch_size=32,
                                verbose=0)

print("Entrenamiento terminado.")

# Predecimos las probabilidades para cada clase
y_pred_prob = model_basic.predict(X_test_log)  # Devuelve una probabilidad por clase para cada muestra

# Convertimos las probabilidades en etiquetas predichas (la clase con mayor probabilidad)
y_pred_class = np.argmax(y_pred_prob, axis=1)  # Selecciona el índice (clase) con mayor probabilidad


**¿Qué hace este modelo?**

- Toma las variables como entrada (X)
- Usa una capa con 16 neuronas y activación ReLU
- Devuelve una predicción por clase (softmax)

Puedes probar aumentando las neuronas o cambiando el optimizador (el mas comun es `adam`) para encontrar el mejor resultado.

In [ ]:
# Para los casos de clasificación multiclase debemos usar el classification_report
# adaptado para reflejar en él todas las clases.
from sklearn.metrics import classification_report

# y_test_mc e y_pred_mc deberían ser etiquetas de clasificación multiclase
# Reemplaza con tus propias variables si usas otro modelo
# digits = 3 indica que tenemos 3 clases
print(classification_report(y_test_log, y_pred_log, digits=3))


### 6.1.3 Red Neuronal Multicapa para Clasificación Binaria

#### Función de pérdida más comun en clasificación binaria

**`binary_crossentropy`**  
- Para clasificación binaria (2 clases)  
- Calcula el error entre la clase real y la probabilidad predicha  
- Recuerda usar la funcion de activación `sigmoid` en la capa de salida
- `model.compile(optimizer='adam', loss='binary_crossentropy')`

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense  # Capa densamente conectada (fully connected)

# Crear el modelo secuencial: se añaden las capas en orden
model_bin = Sequential()

# Primera capa oculta con 16 neuronas y activación ReLU
# input_shape define el número de características de entrada
model_bin.add(Dense(16, activation='relu', input_shape=(X_train_log.shape[1],)))

# Capa de salida con 1 neurona y activación sigmoid
# La salida será un valor entre 0 y 1 (probabilidad de clase 1)
model_bin.add(Dense(1, activation='sigmoid'))

# Compilar el modelo:
# - binary_crossentropy porque es clasificación binaria
# - Métrica de evaluación: accuracy
model_bin.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
# - validation_split=0.2: usa el 20% de los datos como validación
# - epochs=100: número de iteraciones completas sobre los datos
# - batch_size=32: número de muestras procesadas por paso
# - verbose=0: silencia la salida (usa 1 o 2 para verla)
history_bin = model_bin.fit(X_train_log, y_train_log,
                            validation_split=0.2,
                            epochs=100,
                            batch_size=32,
                            verbose=0)

print("Entrenamiento terminado.")

# .predict() devuelve la probabilidad de que la clase sea 1
y_pred_prob_bin = model_bin.predict(X_test_log)

# Aplicamos un umbral de 0.5 para convertir la probabilidad en clase (0 o 1)
# Podemos ajustar este umbral si tenemos datos desbalanceados (0.45, 0.3, ...)
y_pred_class_bin = (y_pred_prob_bin > 0.5).astype(int)

# Mostramos las primeras predicciones
print("Predicciones:", y_pred_class_bin[:5].flatten())


### 6.2 MLP profunda con Dropout

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Crear modelo secuencial (las capas se apilan en orden)
model_deep = Sequential()

# Primera capa densa con 64 neuronas y activación ReLU
# input_shape indica el número de variables de entrada (características)
model_deep.add(Dense(64, activation='relu', input_shape=(X_train_log.shape[1],)))

# Dropout apaga aleatoriamente el 30% de las neuronas durante el entrenamiento
# Esto ayuda a prevenir overfitting (que el modelo memorice demasiado los datos)
model_deep.add(Dropout(0.3))

# Segunda capa oculta con 32 neuronas y activación ReLU
model_deep.add(Dense(32, activation='relu'))

# Capa de salida con softmax para clasificación multiclase
# El número de neuronas depende del número de clases (columnas de y_train_cat)
model_deep.add(Dense(y_train_cat.shape[1], activation='softmax'))

# Compilar el modelo:
# - optimizador: Adam (ajusta los pesos automáticamente)
# - función de pérdida: categorical_crossentropy (clasificación multiclase con one-hot)
# - métrica: accuracy (porcentaje de aciertos)
model_deep.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenamiento del modelo:
# - validation_split: usa el 20% de los datos como validación
# - epochs: número de iteraciones completas sobre el conjunto de datos
# - verbose=0: no muestra la salida del entrenamiento (se puede usar 1 o 2 para ver el progreso)
history_deep = model_deep.fit(X_train_log, y_train_cat, validation_split=0.2, epochs=50, verbose=0)


**¿Qué mejora esta red respecto a la anterior?**

- Tiene más capas y más neuronas.
- `Dropout` apaga aleatoriamente neuronas durante el entrenamiento (evita que el modelo memorice demasiado).
- Es más robusta y generaliza mejor. Reduce el riesgo de overfitting


### 6.3 LSTM - Red para Series Temporales

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical

# Simulamos datos secuenciales: 100 muestras, cada una con 10 pasos de tiempo y 5 variables por paso
X_seq = np.random.rand(100, 10, 5)  # Forma: (n_muestras, pasos_temporales, n_variables)

# Etiquetas aleatorias binarias (0 o 1), codificadas como one-hot para clasificación binaria
y_seq = to_categorical(np.random.randint(0, 2, 100))

# Crear modelo secuencial
model_lstm = Sequential()  # Modelo lineal: capa tras capa en orden

# Añadir una capa LSTM con 32 neuronas
# Esta capa recibe secuencias de 10 pasos con 5 características cada uno
model_lstm.add(LSTM(32, input_shape=(10, 5)))

# Capa de salida con softmax para clasificación binaria (2 clases → one-hot)
model_lstm.add(Dense(2, activation='softmax'))

# Compilar el modelo usando función de pérdida multiclase
model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo con los datos secuenciales simulados
model_lstm.fit(X_seq, y_seq, epochs=10, verbose=0)

print("Modelo LSTM entrenado para datos secuenciales.")

# Realizar predicciones de probabilidad por clase
y_pred_lstm_prob = model_lstm.predict(X_seq)  # X_seq debe tener forma (muestras, pasos_tiempo, variables)

# Convertir probabilidades en etiquetas de clase (la más probable)
y_pred_lstm_class = np.argmax(y_pred_lstm_prob, axis=1)

# Mostrar predicciones y clases reales
print("Predicciones (clase más probable):", y_pred_lstm_class[:5])
print("Clases reales:", np.argmax(y_seq, axis=1)[:5])  # Si y_seq está en formato one-hot

**¿Cuándo usar LSTM?**

- Cuando tienes secuencias temporales: sensores en el tiempo, texto, series temporales
- El modelo "recuerda" estados anteriores


### 6.4 CNN - Red Convolucional para Imágenes o Datos Espaciales

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.models import Sequential

# Simular imágenes (100 muestras de 28x28 píxeles con 1 canal → imágenes escala de grises)
X_img = np.random.rand(100, 28, 28, 1)
y_img = to_categorical(np.random.randint(0, 3, 100))  # Crea etiquetas aleatorias en 3 clases (codificadas one-hot)

# Crear el modelo secuencial
model_cnn = Sequential()  # Modelo secuencial: las capas se añaden en orden lineal

# Capa convolucional 2D: extrae características espaciales (bordes, texturas, etc.)
model_cnn.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))

# Capa de max pooling: reduce la dimensionalidad espacial manteniendo lo más relevante
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

# Capa Flatten: convierte la salida 2D anterior en un vector 1D para conectarla a capas densas
model_cnn.add(Flatten())

# Capa oculta totalmente conectada (fully connected)
model_cnn.add(Dense(64, activation='relu'))

# Capa de salida con softmax: devuelve una probabilidad para cada una de las 3 clases
model_cnn.add(Dense(3, activation='softmax'))

# Compilar el modelo con pérdida para clasificación multiclase
model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo con los datos simulados
model_cnn.fit(X_img, y_img, epochs=10, verbose=0)

print("Modelo CNN entrenado para datos con estructura espacial.")

# Predecimos las probabilidades por clase para cada imagen
# X_img debe tener forma (n_muestras, alto, ancho, canales)
y_pred_cnn_prob = model_cnn.predict(X_img)  # Devuelve una probabilidad por clase para cada muestra

# Convertimos las probabilidades en etiquetas predichas (la clase con mayor probabilidad)
y_pred_cnn_class = np.argmax(y_pred_cnn_prob, axis=1)

# Mostrar predicciones y clases reales
print("Predicciones (clase más probable):", y_pred_cnn_class[:5])
print("Clases reales:", np.argmax(y_img, axis=1)[:5])  # Si y_img está en formato one-hot

**¿Para qué sirve?**

- Para imágenes, series espectrales, o cualquier dato con estructura bidimensional
- Detecta patrones locales con filtros convolucionales


### 6.5 Uso de EarlyStopping para evitar overfitting

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping  # Detiene el entrenamiento si no mejora en validación

# Crear un callback de EarlyStopping:
# - monitor: métrica que se vigila (val_loss = pérdida en el conjunto de validación)
# - patience: nº de épocas que se permite sin mejora antes de detener el entrenamiento
# - restore_best_weights: restaura los pesos del modelo con mejor rendimiento
early_stop = EarlyStopping(monitor='val_loss',
                           patience=5,
                           restore_best_weights=True)

# Definir un modelo secuencial simple con 2 capas ocultas y softmax en salida
model_es = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_log.shape[1],)),  # Primera capa oculta
    Dense(32, activation='relu'),                                       # Segunda capa oculta
    Dense(y_train_cat.shape[1], activation='softmax')                   # Capa de salida (multiclase)
])

# Compilar el modelo:
# - Optimizador: Adam
# - Pérdida: categorical_crossentropy (porque estamos usando codificación one-hot)
# - Métrica: accuracy
model_es.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo:
# - validation_split=0.2: usa el 20% para validación interna
# - epochs=100: máximo de épocas (EarlyStopping puede detener antes)
# - batch_size=32: muestras usadas por iteración
# - callbacks: ejecuta EarlyStopping si no mejora la pérdida de validación
# - verbose=0: no imprime el progreso (usa 1 si quieres ver más)
model_es.fit(X_train_log, y_train_cat,
             validation_split=0.2,
             epochs=100,
             batch_size=32,
             callbacks=[early_stop],
             verbose=0)

print("Entrenamiento con EarlyStopping finalizado.")

**¿Qué hace EarlyStopping?**

- Supervisa el rendimiento en validación
- Si no mejora durante varias épocas (`patience`), detiene el entrenamiento
- Ahorra tiempo y evita sobreajuste


### 6.6 Visualización de la pérdida durante el entrenamiento

Con este código podemos graficar de manera sencilla la pérdida durante el entrenamiento tanto en train como en val. Esto nos ayuda a identificar cuando el modelo está cayendo en  overfitting e identificar como el modelo está mejorando.

In [ ]:
# Visualizar cómo evoluciona la pérdida durante el entrenamiento
plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.title('Evolución de la pérdida durante el entrenamiento')
plt.legend()
plt.grid(True)
plt.show()

## 7. Buenas prácticas de programación y consejos para aprender Machine Learning

Aprender a programar y resolver problemas de Machine Learning lleva tiempo. Estos consejos te ayudarán a desarrollar buenos hábitos desde el inicio y evitar errores comunes.


### 7.1 Estructura y claridad en tu código

- Escribe tu código en bloques lógicos separados por secciones.
- Usa nombres de variables descriptivos (`X_train`, `y_test`, `modelo_rf`, etc).
- Deja comentarios claros explicando qué hace cada bloque.
- Elimina código muerto o duplicado que no se esté usando.


### 7.2 Prueba tu código por partes

- No intentes resolver todo en una sola celda o paso.
- Ejecuta paso a paso: primero carga de datos, luego separación, luego modelo...
- Si algo falla, imprime los shapes de tus variables y revisa el contenido con `.head()`.


### 7.3 Aprende a leer documentación

- Usa `Shift + Tab` en notebooks o `help(función)` para ver qué hace un método.
- Lee la documentación oficial de Scikit-learn, Pandas, Seaborn y Keras.
- Copia ejemplos pequeños y prueba cambiando parámetros.


### 7.4 Reutiliza tus propios códigos y este manual como plantillas

- Guarda notebooks que ya te han funcionado.
- Usa una plantilla para cargar datos, otra para entrenar modelos, otra para gráficas.
- Esto te permitirá resolver ejercicios más rápido en el futuro y te dará confianza.


### 7.5 Ten paciencia y repite lo básico muchas veces

- No intentes dominar todos los modelos a la vez.
- Domina primero uno (RandomForest o Regresión Logística) y evalúalo bien.
- El aprendizaje real viene de repetir un problema con distintos datasets y estructuras.
